In [1]:
# imports

import numpy as np
from PIL import Image
import os

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import VGG16
from keras.layers import Conv2D, MaxPooling2D

In [2]:
# Create Generator

image_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

In [3]:
# Create Data Loaders
train_loader = image_generator.flow_from_directory(
        directory = "Untouch_DataSet_1/Processed_Dataset/train",
        target_size =(224,224),
        batch_size = 32,
        shuffle = True
)

test_loader = image_generator.flow_from_directory(
        directory = "Untouch_DataSet_1/Processed_Dataset/test",
        target_size =(224,224),
        batch_size = 32,
        shuffle = True
)

Found 43567 images belonging to 5 classes.
Found 10898 images belonging to 5 classes.


In [4]:
train_loader.class_indices

{'anthracnose_1': 0,
 'black_spot_1': 1,
 'phytophthora_1': 2,
 'powdery_mildew_1': 3,
 'ring_spot_1': 4}

In [5]:
idx_to_class = {val:key for key, val in dict(train_loader.class_indices).items()}
idx_to_class

{0: 'anthracnose_1',
 1: 'black_spot_1',
 2: 'phytophthora_1',
 3: 'powdery_mildew_1',
 4: 'ring_spot_1'}

In [6]:
# define Model

model = Sequential()
model.add(VGG16(include_top = False, pooling ="avg"))
model.add(Dense(5, activation = "softmax"))

In [7]:
model.layers[0].trainable = False

In [8]:
# train and Test Step
batch_size = 32
steps_train = len(train_loader)
steps_test = len(test_loader)

In [9]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics=["accuracy"])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 5)                 2565      
                                                                 
Total params: 14,717,253
Trainable params: 2,565
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
#VGG16
history = model.fit(
    train_loader,
    epochs = 5
)

Epoch 1/5
1362/1362 [==============================] - 7291s 5s/step - loss: 0.2185 - accuracy: 0.9422
Epoch 2/5
1362/1362 [==============================] - 7894s 6s/step - loss: 0.0117 - accuracy: 0.9974
Epoch 3/5
1051/1362 [======================>.......] - ETA: 27:53 - loss: 0.0056 - accuracy: 0.9989

In [ ]:
test_loss,test_acc = model.evaluate(test_loader, verbose=2)
print('\Test Accuracy : ', test_acc)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# plot the accuracy
plt.plot(history.history['loss'], label='Loss')
plt.legend()
plt.show()
plt.savefig('Loss Acc')

In [ ]:
model.save('Untouch_DataSet_1\Saved_model1\VGG16_PD_Apr09.h5')

In [ ]:
from sklearn.metrics import classification_report

Y_pred = model.predict(train_loader)
y_pred = np.argmax(Y_pred, axis=1)

print(classification_report(train_loader.classes, y_pred))